In [1]:
import cv2 as cv

In [11]:
img = cv.imread('q5Y4XFYojAc.jpg', 1)


temp = cv.Canny(img, 100, 200)


ret,thresh = cv.threshold(temp,127,255,0)
im2, contours, hierarchy = cv.findContours(thresh,cv.RETR_TREE,cv.CHAIN_APPROX_SIMPLE)

cv.imshow("canny", temp)
cv.imshow("result_img", im2)
cv.waitKey(0)
cv.destroyAllWindows()